### Deep dive into Agents in LangChain

- RAG as tool
- ReAct Agent
- ReAct with custom tools
- Agents in depth in LangChain
- Self Ask with Search Agents


In [ ]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
os.environ["LANGCHAIN_PROJECT"] = "langchain_agents"

In [ ]:
# A new way to initialize the models
from langchain.chat_models import init_chat_model

openai_model = init_chat_model(model="gpt-4.1-mini")

In [25]:
from langchain.messages import AIMessage, HumanMessage, SystemMessage

messages = [
    SystemMessage("You are helpful code assistant."),
    HumanMessage("What is Python?"),
    AIMessage("Python is an interpreted programming language."),
    HumanMessage("When was it released?"),
]

response = openai_model.invoke(messages)

print(response.content)

Python was first released in 1991 by Guido van Rossum.


### Predefined tools

#### Wikipedia tool

In [7]:
# Import the tool from the new 'langchain_community' package
from langchain_community.tools import WikipediaQueryRun 
from langchain_community.utilities import WikipediaAPIWrapper # Also moved to community

wiki_api_wrapper = WikipediaAPIWrapper()
tool = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)

print(tool.invoke({"query": "LangGraph"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: C (programming language)
Summary: C is a general-purpose programming language. It was created in the 1970s by Dennis Ritchie and remains widely used and influential. By design, C gives the programmer relatively direct access to the features of the typical CPU architecture, customized for the target instruction set. It has been and continues to be used to implement operating systems (especially kernels), device drivers, and protocol stacks, but its use in application software has been decreasing. C is used on computers that range from the largest supercomputers to the smallest microcontrollers and embedded systems.
A successor to the pro

#### Youtube tool

In [10]:
from langchain_community.tools import YouTubeSearchTool

yt_tool = YouTubeSearchTool()

print(yt_tool.invoke({"query": "Machine Learning"}))

['https://www.youtube.com/watch?v=ukzFI9rgwfU&pp=ygUQTWFjaGluZSBMZWFybmluZw%3D%3D', 'https://www.youtube.com/watch?v=qYNweeDHiyU&pp=ygUQTWFjaGluZSBMZWFybmluZw%3D%3D']


#### Tavily Search

In [11]:
from langchain_tavily import TavilySearch

tavily_tool = TavilySearch()

print(tavily_tool.invoke({"query": "What happened in IPL 2025"}))

{'query': 'What happened in IPL 2025', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.olympics.com/en/news/royal-challengers-bengaluru-ipl-2025-trophy-parade-stampede', 'title': "RCB's IPL 2025 trophy parade takes tragic turn as stampede claims ...", 'content': "# Triumph turns to tragedy as stampede claims lives during RCB’s IPL 2025 victory parade in Bengaluru **Royal Challengers Bengaluru**'s celebrations to mark their **IPL 2025** triumph took a tragic turn on Wednesday as 11 people lost their lives and over 30 were left injured in a stampede during the trophy parade outside the M Chinnaswamy Stadium. Lakhs of people converged at the M Chinnaswamy Stadium to celebrate RCB’s first-ever **Indian Premier League** triumph. In the aftermath of the tragic incident, RCB released a statement, urging supporters to be safe and amending their celebration program. “RCB mourns the tragic loss of lives and extends our heartfelt condolences to the affe

### Custom Tools

- These are very important, where there are no predefined tools
- A function with a tool decorator will become a tool

#### Word Length Tool

In [13]:
from langchain.tools import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word"""
    return len(word)

In [14]:
# To run the tool, use the `invoke` method
get_word_length.invoke({"word": "hello"})

5

#### Multiply tool

In [15]:
@tool
def multiply_nums(num1: int, num2: int) -> int:
    """Multiplies two numbers"""
    return num1 * num2

@tool
def add_nums(num1: int, num2: int) -> int:
    """Adds two numbers"""
    return num1 + num2


In [16]:
print(multiply_nums.invoke({"num1": 2, "num2": 3}))
print(add_nums.invoke({"num1": 2, "num2": 3}))

6
5


In [17]:
# print information about the tool - similar to 
print(multiply_nums.name)
print(multiply_nums.description)
print(multiply_nums.args)

print(add_nums.name)
print(add_nums.description)
print(add_nums.args)


multiply_nums
Multiplies two numbers
{'num1': {'title': 'Num1', 'type': 'integer'}, 'num2': {'title': 'Num2', 'type': 'integer'}}
add_nums
Adds two numbers
{'num1': {'title': 'Num1', 'type': 'integer'}, 'num2': {'title': 'Num2', 'type': 'integer'}}


### Create simple agents

In [20]:
from langchain.tools import tool
from langchain.agents import create_agent
import requests

@tool
def get_weather(city: str) -> str:
    """Returns the weather of a givencity"""
    response = requests.get(f"https://wttr.in/{city}?format=j1")
    return response.json()

agent = create_agent(
    openai_model, 
    tools=[get_weather], 
    system_prompt="You are helpful weather agent, give consize description of the weather"
)

In [24]:
response = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "what is the weather in New York?"
        }
    ]
})

response['messages'][-1].content

"The current weather in New York is sunny with a temperature of 6°C (43°F), feeling like 2°C (35°F). The sky is clear with 0% cloud cover, humidity at 43%, and visibility of 16 km (9 miles). Winds are coming from the west at 27 km/h (17 mph). There is no precipitation, and the pressure is 1014 mb. Overall, it's a clear, crisp day with moderate wind."